# TF IDF creation with package and without package

###### 1.Write a python program to
###### a) show the implementation of a concurrent depth-first crawler (No. of threads =5 and depth = 5).
###### b) Develop the crawler program to handle various challenges (such as Parsing,
###### Stemming, Lemmitization, Link Extraction, Canonicalization, Spider Trap etc.)faced by crawler while implementing.
###### c) Based on the contents retrieved, prepare one inverted index file (with proper representation).

In [36]:
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [37]:
import threading
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
import math

In [38]:
a=[]
l=[]
visited=[]
b=[]
tokens=[]
lem = WordNetLemmatizer()
stop_words=set(stopwords.words('english'))
d={}
def canonical_url(u):
    u = u.lower()
    if u.startswith("http://"):
        u = u[7:]
    if u.startswith("www."):
        u = u[4:]
    if u.endswith("/"):
        u = u[:-1]
    return u

In [39]:
def crawl(lock):
  lock.acquire()
  URL = "https://en.wikipedia.org/wiki/Data_mining"
  counter=1
  l.append(URL)
  for counter in range(0,20):
      i=l.pop(0)
      page = requests.get(i)
      visited.append(i)
      try:       
          soup = BeautifulSoup(page.content, 'html.parser')
          for link in soup.findAll('a', attrs={'href': re.compile("^http://")}):
              if link.get('href') not in visited and canonical_url(link.get('href')) not in d.keys():
                  l.append(link.get('href'))
                  d[canonical_url(link.get('href'))]=link.get('href')
      except:
          continue
  lock.release()

In [40]:
def rep(lock):
  lock.acquire()
  c=0
  for i in visited:
      URL = i
      try:
          page=requests.get(URL)
          soup = BeautifulSoup(page.content, 'html.parser')
          n=str(c)+".txt"
          f=open(n,'w')
          f.write(soup.text)
          a.append(soup.text)
          f.close
          c+=1
      except:
          continue
  lock.release()

In [41]:
def fil(lock):
  global tokens
  lock.acquire()
  for i in a:
    f=word_tokenize(i)
    x=[lem.lemmatize(w).lower() for w in f if not w in stop_words and w.isalpha()]
    x = [j for j in x if j]
    tokens=tokens+x 
    x=' '.join(x)
    b.append(x)
  tokens=list(set(tokens))
  lock.release()

In [42]:
lock = threading.Lock()
t1 = threading.Thread(target=crawl,args=(lock,)) 
t2 = threading.Thread(target=rep,args=(lock,)) 
t3 = threading.Thread(target=fil,args=(lock,))
t1.start()
t1.join()
t2.start()
t3.start()
t2.join()
t3.join()

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [43]:
invert=pd.DataFrame()
tokens=sorted(tokens)
invert['terms']=tokens
for k in range(0,len(visited)):
  line=0
  n=str(k)+'.txt'
  file = open(n, encoding='utf8') 
  read = file.read() 
  file.seek(0) 
  for word in read: 
    if word == '\n': 
        line += 1
  array = [] 
  for i in range(line): 
      array.append(file.readline()) 
  punc = '''!()-[]{};:'"\, <>./?@#$%^&*_~'''
  for ele in read:   
      if ele in punc:   
          read = read.replace(ele, " ")
  read=read.lower()
  for i in range(1): 
    text_tokens = word_tokenize(read) 
  tokens_without_sw = [ word for word in text_tokens if not word in stop_words and word.isalpha()]
  dict = {} 
  
  for i in range(line): 
      check = array[i].lower() 
      for item in tokens: 
    
          if item in check: 
              if item not in dict: 
                  dict[item] = [] 
    
              if item in dict: 
                  dict[item].append(i+1)
          else:
            if item not in dict: 
                  dict[item] = []
            else:
              continue
  if len(dict.values())>0:
    d="doc"+str(k)
    invert[d]=dict.values()
invert.head()

,terms,doc0,doc1,doc2,doc3,doc4,doc5,doc7,doc8,doc9,doc10,doc11,doc12,doc13,doc14,doc15,doc16,doc17,doc18
0,a,"[5, 6, 7, 8, 9, 10, 39, 41, 45, 46, 47, 48, 50...","[13, 16, 18, 19, 20, 21, 22, 24, 26, 27, 28, 3...","[48, 49, 50, 51, 54, 56, 57, 58, 59, 60, 63, 6...","[18, 19, 23, 24, 25, 26, 27, 31, 33, 35, 37, 3...","[3, 18, 19]","[5, 36, 44, 50, 51, 55, 56, 57, 59, 60, 67, 68...","[4, 15, 16, 17, 18, 33, 54, 55, 56, 58, 59, 60...","[28, 50, 51, 52, 53, 56, 58, 59, 60, 61, 62, 6...","[4, 16, 17, 18, 19, 34, 55, 56, 57, 59, 60, 61...","[4, 16, 17, 18, 19, 34, 55, 56, 57, 59, 60, 61...","[4, 16, 17, 18, 19, 34, 55, 56, 57, 59, 60, 61...","[4, 16, 17, 18, 19, 34, 55, 56, 57, 59, 60, 61...","[10, 51, 52, 53, 54, 57, 58, 59, 60, 62, 63, 6...","[7, 12, 20, 21, 22, 24, 27, 28, 31, 32, 33, 37...","[5, 48, 49, 50, 52, 55, 56, 57, 58, 61, 64, 65...","[8, 9, 10, 13, 16, 17, 18, 19, 21, 24, 26, 29,...","[27, 47, 48, 49, 50, 53, 55, 56, 57, 58, 59, 6...",[]
1,aa,"[213, 296, 297, 418, 490, 503, 787]",[],[],[144],[],"[253, 272, 344, 379, 424, 430, 472, 478, 518, ...","[424, 501]",[],[],[],[],[],"[88, 382, 385, 1289, 1292]",[171],[],[],[],[]
2,aaa,[213],[],[],[],[],"[1804, 4632, 4690]",[424],[],[],[],[],[],[],[171],[],[],[],[]
3,aaai,[213],[],[],[],[],[4690],[424],[],[],[],[],[],[],[171],[],[],[],[]
4,aahc,[296],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]


In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer 
tfidf_vectorizer=TfidfVectorizer(use_idf=True) 
tfidf_vectorizer_vectors=tfidf_vectorizer.fit_transform(b)
df=pd.DataFrame(index=tfidf_vectorizer.get_feature_names())
for i in range(len(a)):
  n='tfidf'+str(i+1)
  first_vector_tfidfvectorizer=tfidf_vectorizer_vectors[i] 
  df[n]=first_vector_tfidfvectorizer.T.todense()
  df.sort_values(by=[n],ascending=False)
df.head()

,tfidf1,tfidf2,tfidf3,tfidf4,tfidf5,tfidf6,tfidf7,tfidf8,tfidf9,tfidf10,tfidf11,tfidf12,tfidf13,tfidf14,tfidf15,tfidf16,tfidf17,tfidf18,tfidf19,tfidf20
aa,0.000000,0.0,0.0,0.0,0.0,0.030364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.033979,0.0,0.0,0.0,0.0,0.0,0.0
aaa,0.000000,0.0,0.0,0.0,0.0,0.004935,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
aaai,0.005431,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
aahc,0.005431,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
aaooin,0.000000,0.0,0.0,0.0,0.0,0.004935,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


#2.Write a python program to show the implementation of Golomb Encoding-decoding technique.

1.Encode


In [45]:
from math import log,ceil
log2 = lambda x: log(x,2)

def binary(x,l=1):
  fmt = '{0:0%db}' % l 
  return fmt.format(x)
def unary(x):
  return x*'1'+'0'
def golomb(b, x):
  q = int((x) / b)
  r = int((x) % b)
  l = int(ceil(log2(b)))
  return unary(q) + binary(r, l)
x=int(input("Enter the X: "))
b=int(input("Enter the b: "))
print(golomb(b,x))

Enter the X: 25
Enter the b: 11
1100011


In [46]:
from math import log,ceil
log2 = lambda x: log(x,2)

def binary(x,l=1):
  fmt = '{0:0%db}' % l 
  return fmt.format(x)
def unary(x):
  return x*'1'+'0'
def golomb(b, x):
  q = int((x) / b)
  r = int((x) % b)
  l = int(ceil(log2(b)))
  return unary(q) + binary(r, l)
x=int(input("Enter the X: "))
b=int(input("Enter the b: "))
print(golomb(b,x))

Enter the X: 25
Enter the b: 16
101001


In [47]:
from math import log,ceil
log2 = lambda x: log(x,2)

def binary(x,l=1):
  fmt = '{0:0%db}' % l 
  return fmt.format(x)
def unary(x):
  return x*'1'+'0'
def golomb(b, x):
  q = int((x) / b)
  r = int((x) % b)
  l = int(ceil(log2(b)))
  return unary(q) + binary(r, l)
x=int(input("Enter the X: "))
b=int(input("Enter the b: "))
print(golomb(b,x))

Enter the X: 37
Enter the b: 11
11100100


In [48]:
from math import log,ceil
log2 = lambda x: log(x,2)

def binary(x,l=1):
  fmt = '{0:0%db}' % l 
  return fmt.format(x)
def unary(x):
  return x*'1'+'0'
def golomb(b, x):
  q = int((x) / b)
  r = int((x) % b)
  l = int(ceil(log2(b)))
  return unary(q) + binary(r, l)
x=int(input("Enter the X: "))
b=int(input("Enter the b: "))
print(golomb(b,x))

Enter the X: 37
Enter the b: 16
1100101


2.Decode

In [49]:
import math

def decode(x):
    num=0;
    for i in range(len(x)):
        num+=(int(x[len(x)-1-i])*(math.pow(2,i)));
    return num;

x=str(input('Enter code: '))
x=list(x)
b=int(input('Enter value of b: '))
i=math.floor(math.log(b,2))
d=math.pow(2,i+1)-b


p2=0;
l=1;
while(p2<len(x)):
    t=0;
    flag=0;
    r=[];
    k=i;
    q=0;
    for p in range(p2,len(x)):
        if(x[p]=='0' and flag==0):
            t+=1;
            continue;
        if(x[p]=='1' and flag==0):
            q=t;
            flag=1;
            continue;
        r.append(x[p]);
        k-=1;
        if(k==0):
            rnum=decode(r);
            if(rnum<d):
                p2=p+1;
                break;
        if(k==-1):
            rnum=decode(r);
            rnum=rnum-d;
            p2=p+1;
            break;
    ans=q*b+rnum;
    print(ans);
    l=0;

Enter code: 1111111110010001101
Enter value of b: 10
9.0
8.0
10.0
5.0


# 3. Write a python program to extract the contents (excluding any tags) from two websites
###### https://en.wikipedia.org/wiki/Web_mining 
###### https://en.wikipedia.org/wiki/Data_mining
###### Save the content in two separate files. Construct a trie based on the content retrieved in using HashMap / B-Tree / Dictionary. Write a program to show the implementation of Predictive Typing and Auto-Correct using the trie prepared.

In [50]:
URL = "https://en.wikipedia.org/wiki/Web_mining"
page=requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')
f=open('web.txt','w')
web=soup.text
f.write(soup.text)
f.close
URL = "https://en.wikipedia.org/wiki/Data_mining"
page=requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')
f=open('data.txt','w')
data=soup.text
f.write(soup.text)
f.close

<function TextIOWrapper.close>

In [51]:
f=word_tokenize(web)
f=[w for w in f if w.isalpha()]
x=[lem.lemmatize(w) for w in f if not w in stop_words and w.isalpha()]
x = [j for j in x if j] 
w=x
f=word_tokenize(data)
f=[w for w in f if w.isalpha()]
x=[lem.lemmatize(w) for w in f if not w in stop_words and w.isalpha()]
x = [j for j in x if j]
w+=x
w=list(set(w))

In [52]:
class TrieNode(): 
	def __init__(self): 
		self.children = {} 
		self.last = False

class Trie(): 
	def __init__(self): 
		self.root = TrieNode() 
		self.word_list = [] 

	def formTrie(self, keys): 
		for key in keys: 
			self.insert(key)

	def insert(self, key):  
		node = self.root 

		for a in list(key): 
			if not node.children.get(a): 
				node.children[a] = TrieNode() 
			node = node.children[a] 
		node.last = True

	def search(self, key): 
		node = self.root 
		found = True

		for a in list(key): 
			if not node.children.get(a): 
				found = False
				break

			node = node.children[a] 

		return node and node.last and found 

	def suggestionsRec(self, node, word): 
		if node.last: 
			self.word_list.append(word) 

		for a,n in node.children.items(): 
			self.suggestionsRec(n, word + a) 

	def printAutoSuggestions(self, key): 
		node = self.root 
		not_found = False
		temp_word = '' 

		for a in list(key): 
			if not node.children.get(a): 
				not_found = True
				break

			temp_word += a 
			node = node.children[a] 

		if not_found: 
			return 0
		elif node.last and not node.children: 
			return -1

		self.suggestionsRec(node, temp_word) 

		for s in self.word_list: 
			print(s) 
		return 1
status = ["Not found", "Found"] 
t = Trie() 
t.formTrie(w) 
x=input("enter some letter:  ")
comp = t.printAutoSuggestions(x) 
if comp == -1: 
	print("No other strings found with this prefix\n") 
elif comp == 0: 
	print("No string found with this prefix\n") 

enter some letter:  min
mining
miningFormal
miningWorld
minimization
mine
miner
mined


# 4. Write a python program to extract the contents (excluding any tags) from the following five websites
1.   https://en.wikipedia.org/wiki/Web_mining
2.   https://en.wikipedia.org/wiki/Data_mining
3.   https://en.wikipedia.org/wiki/Artificial_intelligence
4.   https://en.wikipedia.org/wiki/Machine_learning
5.   https://en.wikipedia.org/wiki/Mining
###### Refined the contents by applying stopword removal and lemmatization process. Save the refined tokenized content in five separate files. Considering a vector space model and do the following operations according to the query “Mining large volume of data”.



In [53]:
d=[]
extra=[]
docs=[]
URL = ["https://en.wikipedia.org/wiki/Web_mining","https://en.wikipedia.org/wiki/Data_mining","https://en.wikipedia.org/wiki/Artificial_intelligence","https://en.wikipedia.org/wiki/Machine_learning","https://en.wikipedia.org/wiki/Mining"]
for i in URL:
  page=requests.get(i)
  soup = BeautifulSoup(page.content, 'html.parser')
  d.append(soup.text)

In [54]:
y=[]
c=1
for i in d:
    f=word_tokenize(i)
    x=[lem.lemmatize(w).lower() for w in f if not w in stop_words and w.isalpha()]
    x = [j for j in x if j] 
    docs.append(x)
    x=' '.join(x)
    extra.append(x)
    n="Refinedtoken"+str(c)
    c+=1
    f=open(n,"w")
    f.write(x)
    f.close()
for i in docs:
  y+=i
y=list(set(y))
y=sorted(y)

In [55]:
d1={}
d2={}
d3={}
d4={}
d5={}
for k in range(5):
  for i in y:
    c=0
    for j in docs[k]:
      if i==j.lower():
        c+=1
    if k==0:
      d1[i]=c
    elif k==1:
      d2[i]=c
    elif k==2:
      d3[i]=c
    elif k==3:
      d4[i]=c
    else:
      d5[i]=c

Bag Of Words, normalised TF, IDF and TF-IDF:

In [56]:
bag=pd.DataFrame()
bag['DOC']=['d1','d2','d3','d4','d5']
for i in y:
  bag[i]=[d1[i],d2[i],d3[i],d4[i],d5[i]]
bag.head()

,DOC,a,aaa,aaai,aahc,abandoned,abbaszadeh,abduction,ability,able,abnormal,aborigine,about,abrahams,abroad,absence,abstract,abstraction,abundance,abuse,abusing,academic,academy,accelerated,accelerating,acceleration,accelerator,accept,acceptable,acceptance,accepted,access,accessed,accessibility,accessible,accessing,accessscience,accident,accolade,accomplish,...,yielded,yike,ying,yongcan,yooreeka,york,yoshida,yoshikawa,yoshua,you,young,your,yu,yudkowsky,yun,z,zaiane,zairja,zalta,zanasi,zdnet,zdravko,zealand,zed,zenon,zero,zh,zhan,zhang,zheng,zhong,zierahn,zimek,zinc,zombie,zuckerberg,čapek,štiavnica,тарашкевіца,雄踞人类世界排名第一已两年
0,d1,4,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,4,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,d2,16,0,1,1,0,0,0,1,3,0,0,1,0,0,0,0,0,0,0,0,8,0,0,0,1,0,0,1,1,0,4,0,1,0,0,0,0,0,0,...,0,1,0,0,0,3,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,d3,67,1,2,0,2,1,2,11,7,0,0,2,0,0,0,7,1,0,0,0,5,1,1,3,1,0,1,1,0,3,3,0,1,1,0,1,2,0,2,...,0,0,0,1,0,22,1,1,1,1,3,3,1,1,0,0,0,1,1,0,2,0,1,0,1,0,2,0,1,0,0,1,0,0,1,2,3,0,1,1
3,d4,32,0,0,0,1,0,0,4,3,1,0,1,0,0,1,1,0,0,1,0,2,0,0,0,1,1,0,0,0,0,1,0,1,0,0,0,0,0,1,...,1,0,1,0,1,4,0,0,3,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,1,0,1,0,1,0,0,0,0,0,0,0
4,d5,16,0,0,0,0,0,0,0,2,0,1,1,1,2,0,0,0,1,0,1,0,1,0,0,0,0,0,1,1,0,7,1,0,1,0,0,6,1,0,...,0,0,0,0,0,6,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,2,0,0,0,1,1,0


In [57]:
tf1={}
tf2={}
tf3={}
tf4={}
tf5={}
tf=pd.DataFrame()
tf['term']=y
for i in y: 
  tf1[i]=d1[i]/len(docs[0])
  tf2[i]=d2[i]/len(docs[1])
  tf3[i]=d3[i]/len(docs[2])
  tf4[i]=d3[i]/len(docs[3])
  tf5[i]=d3[i]/len(docs[4])
#t=[i/len(extra[0]) for i in d1.values()]
tf['d1']=tf1.values() 
#t=[i/len(extra[1]) for i in d2.values()]
tf['d2']=tf2.values() 
#t=[i/len(extra[2]) for i in d3.values()]
tf['d3']=tf3.values() 
#t=[i/len(extra[3]) for i in d4.values()]
tf['d4']=tf4.values() 
#t=[i/len(extra[4]) for i in d5.values()]
tf['d5']=tf5.values() 

tf.head()

,term,d1,d2,d3,d4,d5
0,a,0.001674,0.003723,0.004235,0.009626,0.009002
1,aaa,0.000000,0.000000,0.000063,0.000144,0.000134
2,aaai,0.000000,0.000233,0.000126,0.000287,0.000269
3,aahc,0.000000,0.000233,0.000000,0.000000,0.000000
4,abandoned,0.000000,0.000000,0.000126,0.000287,0.000269


In [58]:
id1={}
id2={}
id3={}
id4={}
id5={}
idfd={}
for i in y:
  id1[i]=1 if i in extra[0] else 0
  id2[i]=1 if i in extra[1] else 0
  id3[i]=1 if i in extra[2] else 0
  id4[i]=1 if i in extra[3] else 0
  id5[i]=1 if i in extra[4] else 0
idf=pd.DataFrame()
v=[]
for i in y:
  s=sum([id1[i],id2[i],id3[i],id4[i],id5[i]])
  x=math.log(5/s+1)
  v.append(x)
  idfd[i]=x
idf['term']=y
idf['idfv']=v
idf.head()

,term,idfv
0,a,0.693147
1,aaa,1.252763
2,aaai,1.252763
3,aahc,1.791759
4,abandoned,1.252763


In [59]:
tfidf=pd.DataFrame()
tfidf['terms']=y
a1={}
a2={}
a3={}
a4={}
a5={}
for i in y:
  a1[i]=tf1[i]*idfd[i]
  a2[i]=tf2[i]*idfd[i]
  a3[i]=tf3[i]*idfd[i]
  a4[i]=tf4[i]*idfd[i]
  a5[i]=tf5[i]*idfd[i]
tfidf['tfidf1']=a1.values()
tfidf['tfidf2']=a2.values()
tfidf['tfidf3']=a3.values()
tfidf['tfidf4']=a4.values()
tfidf['tfidf5']=a5.values()
tfidf.head()

,terms,tfidf1,tfidf2,tfidf3,tfidf4,tfidf5
0,a,0.00116,0.002580,0.002935,0.006673,0.006240
1,aaa,0.00000,0.000000,0.000079,0.000180,0.000168
2,aaai,0.00000,0.000291,0.000158,0.000360,0.000337
3,aahc,0.00000,0.000417,0.000000,0.000000,0.000000
4,abandoned,0.00000,0.000000,0.000158,0.000360,0.000337


Cosine and Euclidian Similarity:

In [60]:
from itertools import combinations
doc=list(combinations([1,2,3,4,5],2))
cos={}
for i in doc:
  a1='tfidf'+str(i[0])
  a2='tfidf'+str(i[1])
  num=tfidf[a1]*tfidf[a2]
  s=sum(num)
  den=math.sqrt(sum(tfidf[a1]))*math.sqrt(sum(tfidf[a2]))
  v=s/den
  cos[i]=v
cos

{(1, 2): 0.002232251152519334,
 (1, 3): 0.00039645144575271293,
 (1, 4): 0.0005977451709445423,
 (1, 5): 0.0005780250853956587,
 (2, 3): 0.0005869471929411408,
 (2, 4): 0.0008849629732435524,
 (2, 5): 0.0008557673454271286,
 (3, 4): 0.002544929471724906,
 (3, 5): 0.0024609702373592744,
 (4, 5): 0.003710499964067049}

In [61]:
eu={}
for j in doc:
  sv=[]
  a1='tfidf'+str(j[0])
  a2='tfidf'+str(j[1])
  num=tfidf[a1]-tfidf[a2]
  for i in num:
    sv.append(i**2)
  s=sum(sv)
  v=math.sqrt(s)
  eu[j]=v
eu

{(1, 2): 0.05795095149915952,
 (1, 3): 0.06929734743979765,
 (1, 4): 0.10582337028754113,
 (1, 5): 0.10081678787922656,
 (2, 3): 0.064578103413761,
 (2, 4): 0.10040029103650241,
 (2, 5): 0.09540456788695983,
 (3, 4): 0.053525442915478276,
 (3, 5): 0.04732405332469929,
 (4, 5): 0.006201389590779556}

Document ranking:


In [62]:
drc={k: v for k, v in sorted(cos.items(), key=lambda item: item[1],reverse=True)}
for i in drc.keys():
  print(i," : ",drc[i])

(4, 5)  :  0.003710499964067049
(3, 4)  :  0.002544929471724906
(3, 5)  :  0.0024609702373592744
(1, 2)  :  0.002232251152519334
(2, 4)  :  0.0008849629732435524
(2, 5)  :  0.0008557673454271286
(1, 4)  :  0.0005977451709445423
(2, 3)  :  0.0005869471929411408
(1, 5)  :  0.0005780250853956587
(1, 3)  :  0.00039645144575271293


In [63]:
drc={k: v for k, v in sorted(eu.items(), key=lambda item: item[1],reverse=True)}
for i in drc.keys():
  print(i," : ",drc[i])

(1, 4)  :  0.10582337028754113
(1, 5)  :  0.10081678787922656
(2, 4)  :  0.10040029103650241
(2, 5)  :  0.09540456788695983
(1, 3)  :  0.06929734743979765
(2, 3)  :  0.064578103413761
(1, 2)  :  0.05795095149915952
(3, 4)  :  0.053525442915478276
(3, 5)  :  0.04732405332469929
(4, 5)  :  0.006201389590779556


Acoording to Query:

In [64]:
q=input("enter the query: ")
q=word_tokenize(q)
r=[lem.lemmatize(w).lower() for w in q if not w in stop_words and w.isalpha()]
r

enter the query: Mining large volume of data


['mining', 'large', 'volume', 'data']

In [65]:
d1={}
d2={}
d3={}
d4={}
d5={}
for k in range(5):
  for i in r:
    c=0
    for j in docs[k]:
      if i==j.lower():
        c+=1
    if k==0:
      d1[i]=c
    elif k==1:
      d2[i]=c
    elif k==2:
      d3[i]=c
    elif k==3:
      d4[i]=c
    else:
      d5[i]=c

In [66]:
tf1={}
tf2={}
tf3={}
tf4={}
tf5={}
tf=pd.DataFrame()
tf['term']=r
for i in r: 
  tf1[i]=d1[i]/len(docs[0])
  tf2[i]=d2[i]/len(docs[1])
  tf3[i]=d3[i]/len(docs[2])
  tf4[i]=d3[i]/len(docs[3])
  tf5[i]=d3[i]/len(docs[4])
#t=[i/len(extra[0]) for i in d1.values()]
tf['d1']=tf1.values() 
#t=[i/len(extra[1]) for i in d2.values()]
tf['d2']=tf2.values() 
#t=[i/len(extra[2]) for i in d3.values()]
tf['d3']=tf3.values() 
#t=[i/len(extra[3]) for i in d4.values()]
tf['d4']=tf4.values() 
#t=[i/len(extra[4]) for i in d5.values()]
tf['d5']=tf5.values() 

tf.head()

,term,d1,d2,d3,d4,d5
0,mining,0.039331,0.038390,0.000442,0.001006,0.000940
1,large,0.000000,0.002559,0.000758,0.001724,0.001612
2,volume,0.000418,0.000465,0.000253,0.000575,0.000537
3,data,0.025941,0.060726,0.001959,0.004454,0.004165


In [67]:
id1={}
id2={}
id3={}
id4={}
id5={}
idfd={}
for i in r:
  id1[i]=1 if i in extra[0] else 0
  id2[i]=1 if i in extra[1] else 0
  id3[i]=1 if i in extra[2] else 0
  id4[i]=1 if i in extra[3] else 0
  id5[i]=1 if i in extra[4] else 0
idf=pd.DataFrame()
v=[]
for i in r:
  s=sum([id1[i],id2[i],id3[i],id4[i],id5[i]])
  x=math.log(5/s+1)
  v.append(x)
  idfd[i]=x
idf['term']=r
idf['idfv']=v
idf.head()

,term,idfv
0,mining,0.693147
1,large,0.810930
2,volume,0.693147
3,data,0.693147


In [68]:
tfidf=pd.DataFrame()
tfidf['terms']=r
a1={}
a2={}
a3={}
a4={}
a5={}
for i in r:
  a1[i]=tf1[i]*idfd[i]
  a2[i]=tf2[i]*idfd[i]
  a3[i]=tf3[i]*idfd[i]
  a4[i]=tf4[i]*idfd[i]
  a5[i]=tf5[i]*idfd[i]
tfidf['tfidf1']=a1.values()
tfidf['tfidf2']=a2.values()
tfidf['tfidf3']=a3.values()
tfidf['tfidf4']=a4.values()
tfidf['tfidf5']=a5.values()
tfidf.head()

,terms,tfidf1,tfidf2,tfidf3,tfidf4,tfidf5
0,mining,0.027262,0.026610,0.000307,0.000697,0.000652
1,large,0.000000,0.002075,0.000615,0.001398,0.001307
2,volume,0.000290,0.000323,0.000175,0.000398,0.000373
3,data,0.017981,0.042092,0.001358,0.003087,0.002887


##### Cosine and Euclidian Similarity according to query and document ranking

In [69]:
cos={}
for i in doc:
  a1='tfidf'+str(i[0])
  a2='tfidf'+str(i[1])
  num=tfidf[a1]*tfidf[a2]
  s=sum(num)
  den=math.sqrt(sum(tfidf[a1]))*math.sqrt(sum(tfidf[a2]))
  v=s/den
  cos[i]=v
drc={k: v for k, v in sorted(cos.items(), key=lambda item: item[1],reverse=True)}
for i in drc.keys():
  print(i," : ",drc[i])

(1, 2)  :  0.02605347545535981
(2, 4)  :  0.007607009908563435
(2, 5)  :  0.007356048640350983
(2, 3)  :  0.005045310648582261
(1, 4)  :  0.00468186619778377
(1, 5)  :  0.004527407732142214
(1, 3)  :  0.003105223953543746
(4, 5)  :  0.0021019207744874357
(3, 4)  :  0.001441649421378994
(3, 5)  :  0.001394088267725213


In [70]:
eu={}
for j in doc:
  sv=[]
  a1='tfidf'+str(j[0])
  a2='tfidf'+str(j[1])
  num=tfidf[a1]-tfidf[a2]
  for i in num:
    sv.append(i**2)
  s=sum(sv)
  v=math.sqrt(s)
  eu[j]=v
deu={k: v for k, v in sorted(eu.items(), key=lambda item: item[1],reverse=True)}
for i in deu.keys():
  print(i," : ",deu[i])

(2, 3)  :  0.04851047315978039
(2, 5)  :  0.04702601110515685
(2, 4)  :  0.04683270626672092
(1, 3)  :  0.031674953854355925
(1, 5)  :  0.03062096460381193
(1, 4)  :  0.030487380134688782
(1, 2)  :  0.024208739500678637
(3, 4)  :  0.0019508187376542693
(3, 5)  :  0.001724799365291671
(4, 5)  :  0.00022601937236259824
